# Quantitative experiments with Credit Card Approval dataset
This dataset contains information about bank customers, as well as information regarding their debt payments (if any). https://cutt.ly/xQ1mqyo

In [1]:
import sys
sys.path.append('C:\\Users\\iamollas\\Desktop\\Altruist New')
model_path = 'C:\\Users\\iamollas\\Desktop\\Altruist New\\experiments\\quantitative\\Models\\D2\\'
weights_path = 'C:\\Users\\iamollas\\Desktop\\Altruist New\\experiments\\quantitative\\Weights\\D2\\'
data_path = 'C:\\Users\\iamollas\\Desktop\\Altruist New\\experiments\\quantitative\\Preprocessed Data\\D2\\'

In [2]:
import random
import csv
import warnings
import json
import lime.lime_tabular as lt
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense
from keras import Sequential
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, concatenate
import json

import keras
from keras.models import Sequential, Model
from innvestigate.utils.keras import checks
import innvestigate
import innvestigate.utils as iutils
from sklearn.model_selection import train_test_split
from altruist import Altruist
from meta_explain import MetaExplain
from utilities.dataset import Dataset
from sklearn.preprocessing import maxabs_scale
import numpy as np
np.seterr(invalid='ignore')
warnings.filterwarnings("ignore")

Using TensorFlow backend.


We will now load the Credit Approval dataset.

In [3]:
credit = Dataset()
X_res, X, y_res, y, feature_names = credit.load_credit_approval(
    original_data_available=True)
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, stratify=y_res, random_state=42)
class_names = ['Denial', 'Approval']
feature_names = list(feature_names)

Number of observations in final dataset: (35442, 13)
Original dataset shape Counter({0: 34846, 1: 596})
TomekLinks: Resampled dataset shape Counter({0: 34649, 1: 596})
NC: Resampled dataset shape Counter({0: 33196, 1: 596})
NM: Resampled dataset shape Counter({0: 596, 1: 596})
Random: Resampled dataset shape Counter({0: 596, 1: 596})


And we will build two neural network models. One linear and one non-linear, and one uneccessary complex model.

In [4]:
from sklearn.preprocessing import MaxAbsScaler
transformer = MaxAbsScaler().fit(X_train)

X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)

In [5]:
linear_neural = Sequential()
linear_neural.add(Dense(1, activation='sigmoid',
                        input_shape=(len(X_train[0]),)))
linear_neural.compile(optimizer='adam', loss='binary_crossentropy')

neural = Sequential()
neural.add(Dense(160, activation='relu', input_shape=(len(X_train[0]),)))
neural.add(Dense(80, activation='tanh'))
neural.add(Dense(40, activation='tanh'))
neural.add(Dense(20, activation='tanh'))
neural.add(Dense(16, activation='tanh'))
neural.add(Dense(1, activation='sigmoid'))
neural.compile(optimizer='adam', loss='binary_crossentropy')

complex_input = Input(shape=(len(X_train[0]),))
complex_r = []
for i in range(12):
    temp_layer = Dense(units=128, activation='relu')(complex_input)
    temp_layer = Dropout(0.3)(temp_layer)
    temp_layer = Dense(units=64, activation='selu')(temp_layer)
    temp_layer = Dropout(0.3)(temp_layer)
    temp_layer = Dense(units=32, activation='selu')(temp_layer)
    complex_r.append(temp_layer)
complex_r.append(complex_input)
complex = concatenate(complex_r)
complex = Dropout(0.3)(complex)
complex = Dense(160, activation='tanh')(complex)
complex = concatenate([complex, complex_input])
complex = Dropout(0.3)(complex)
complex = Dense(100, activation='tanh')(complex)
complex = Dropout(0.3)(complex)
complex_output = Dense(1, activation='sigmoid')(complex)
complex_neural = Model(complex_input, complex_output)
complex_neural.compile(optimizer="adam", loss=['binary_crossentropy'])

models = {'lNN': linear_neural, 'NN': neural, 'cNN': complex_neural}

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Let's evaluate our models!

In [6]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def compute_scores(name, y_test, y_pred):
    if type(y_pred[0]) == type(np.ndarray([1])):
        y_pred = np.array([1 if i[0] > 0.5 else 0 for i in y_pred])
    print(name)
    print('\t', 'F1:', f1_score(y_test, y_pred, average='macro'))
    print('\t', 'Precision:', precision_score(y_test, y_pred, average='macro'))
    print('\t', 'Recall:', recall_score(y_test, y_pred, average='macro'))
    print('\t', 'Accuracy:', accuracy_score(y_test, y_pred))

In [7]:
train = False
for name, model in models.items():
    if train:
        check_point = ModelCheckpoint(
            "D2_"+name+".hdf5", monitor="val_loss", verbose=0, save_best_only=True, mode="auto")
        model.fit(X_train, y_train, epochs=500, batch_size=32,
                  validation_split=0.2, verbose=0, callbacks=[check_point])
        model.load_weights("D2_"+name+".hdf5")
    else:
        model.load_weights(model_path+"D2_"+name+".hdf5")
    y_pred = model.predict(X_test)
    compute_scores(name, y_test, y_pred)



lNN
	 F1: 0.5983193277310924
	 Precision: 0.5983193277310924
	 Recall: 0.5983193277310924
	 Accuracy: 0.5983263598326359
NN
	 F1: 0.6903277769995797
	 Precision: 0.6904161412358134
	 Recall: 0.6903361344537815
	 Accuracy: 0.6903765690376569
cNN
	 F1: 0.6984861227922625
	 Precision: 0.699220396123051
	 Recall: 0.6986344537815126
	 Accuracy: 0.698744769874477


We will prepare our predict functions to work well with our python scripts!

In [8]:
def predict_cNN(x):
    prediction = models['cNN'].predict(x)
    return [i[0] for i in prediction]

def predict_NN(x):
    prediction = models['NN'].predict(x)
    return [i[0] for i in prediction]

def predict_lNN(x):
    prediction = models['lNN'].predict(x)
    return [i[0] for i in prediction]

predict_functions = {'lNN': predict_lNN, 'NN': predict_NN, 'cNN': predict_cNN}

Let's prepare our explainers!

In [9]:
def analyzer_generators(model):
    Xs = iutils.to_list(model.outputs)
    ret = []
    for x in Xs:
        layer, node_index, tensor_index = x._keras_history
        if checks.contains_activation(layer, activation="sigmoid"):
            if isinstance(layer, keras.layers.Activation):
                ret.append(layer.get_input_at(node_index))
            else:
                layer_wo_act = innvestigate.utils.keras.graph.copy_layer_wo_activation(
                    layer)
                ret.append(layer_wo_act(layer.get_input_at(node_index)))
    modified_model = Model(input=model.input, output=ret)
    modified_model.trainable = False
    modified_model.compile(optimizer="adam", loss=[
                           'binary_crossentropy'], metrics=['accuracy'])
    analyzer_IG = innvestigate.create_analyzer(
        'integrated_gradients', modified_model, reference_inputs=16*[0])
    analyzer_LRP = innvestigate.create_analyzer('lrp.z', modified_model)
    return [analyzer_IG, analyzer_LRP]

In [10]:
analyzer_NN = analyzer_generators(neural)
analyzer_lNN = analyzer_generators(linear_neural)
analyzer_cNN = analyzer_generators(complex_neural)

We prepare each interpretation technique to have the same format!

In [11]:
def fi_lNN(instance, predict_function):
    return [i[0] for i in models['lNN'].get_weights()[0]]

def fi_IG_NN(instance, predict_function):
    return analyzer_NN[0].analyze(np.array([instance]))[0]

def fi_LRP_NN(instance, predict_function):
    return analyzer_NN[1].analyze(np.array([instance]))[0]

def fi_IG_lNN(instance, predict_function):
    return analyzer_lNN[0].analyze(np.array([instance]))[0]

def fi_LRP_lNN(instance, predict_function):
    return analyzer_lNN[1].analyze(np.array([instance]))[0]

def fi_IG_cNN(instance, predict_function):
    return analyzer_cNN[0].analyze(np.array([instance]))[0]

def fi_LRP_cNN(instance, predict_function):
    return analyzer_cNN[1].analyze(np.array([instance]))[0]

explainer = lt.LimeTabularExplainer(training_data=X_train,
                                    feature_names=feature_names, class_names=class_names,
                                    discretize_continuous=True, mode='regression')

def fi_lime(instance, predict_function):
    def predict(x):
        return np.array([i for i in predict_function(x)])
    b = explainer.explain_instance(instance, predict,
                                   num_features=len(list(feature_names)))[0].local_exp
    b = b[list(b.keys())[0]]
    b.sort()
    return [i[1] for i in list(b)]

def fi_random(instance, predict_function):
    seed = (instance[0].sum() +
            instance[0].mean() +
            X_train[0][0] +
            X_train[0][1])/10
    random.seed(seed)
    return [random.randrange(-1000, 1000)/1000 for i in range(len(feature_names))]


We compute and save the interpretations for easier reproducibility

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
for neural_name, neural_type in predict_functions.items():
    if neural_name == 'lNN':
        fi_techniques = [fi_lNN, fi_IG_lNN, fi_LRP_lNN, fi_lime, fi_random]
        fi_names = ['Inherent', 'IG', 'LRP', 'LIME', 'RAND']
    elif neural_name == 'NN':
        fi_techniques = [fi_IG_NN, fi_LRP_NN, fi_lime, fi_random]
        fi_names = ['IG', 'LRP', 'LIME', 'RAND']
    else:
        fi_techniques = [fi_IG_cNN, fi_LRP_cNN, fi_lime, fi_random]
        fi_names = ['IG', 'LRP', 'LIME', 'RAND']

    importance_train = []
    for instance in X_train:
        importance_instance = []
        for fi in fi_techniques:
            importance_instance.append(maxabs_scale(fi(instance, neural_type)))
        importance_train.append(importance_instance)
    importance_train = np.array(importance_train)

    importance_test = []
    for instance in X_test:
        importance_instance = []
        for fi in fi_techniques:
            importance_instance.append(maxabs_scale(fi(instance, neural_type)))
        importance_test.append(importance_instance)
    importance_test = np.array(importance_test)

    importances = {'train': importance_train.tolist(),
                   'test': importance_test.tolist()}
    with open('D2_'+neural_name+'.txt', 'w') as outfile:
        json.dump(importances, outfile)

Our quantitative experiments!

In [ ]:
for neural_name, neural_type in predict_functions.items():
    if neural_name == 'lNN':
        fi_techniques = [fi_lNN, fi_IG_lNN, fi_LRP_lNN, fi_lime, fi_random]
        fi_names = ['Inherent', 'IG', 'LRP', 'LIME', 'RAND']
    elif neural_name == 'NN':
        fi_techniques = [fi_IG_NN, fi_LRP_NN, fi_lime, fi_random]
        fi_names = ['IG', 'LRP', 'LIME', 'RAND']
    else:
        fi_techniques = [fi_IG_cNN, fi_LRP_cNN, fi_lime, fi_random]
        fi_names = ['IG', 'LRP', 'LIME', 'RAND']
    meta_names = ['Average', 'Median', 'RuleBased']

    with open(weights_path+'D2_'+neural_name+'.txt') as json_file:
        importances = json.load(json_file)
    importance_train = np.array(importances['train'])
    importance_test = np.array(importances['test'])
    meta_explain = MetaExplain(importance_train, feature_names)

    print('Starting evaluation for', neural_name)
    for noise in ['weak', 'normal', 'strong']:
        for delta in [0 , 0.0001, 0.001, 0.01, 0.1]:
            my_altruist = Altruist(
                neural_type, X_train, fi_techniques, feature_names, level=noise, delta=delta)
            fis_scores = []
            meta_scores = []
            nzw_scores = []
            nzw_scores_delta = []
            for i in fi_techniques:
                fis_scores.append([])
                nzw_scores.append([])
                nzw_scores_delta.append([])
            for i in meta_names:
                meta_scores.append([])
                nzw_scores.append([])
                nzw_scores_delta.append([])
            for j in range(len(X_test)):
                my_altruist.fis = len(fi_techniques)
                a = my_altruist.find_untruthful(X_test[j], importance_test[j])
                for i in range(len(importance_test[j])):
                    cnzw = 0
                    cnzwt = 0
                    for k in importance_test[j][i]:
                        if abs(k) > 0:
                            cnzw += 1
                        if abs(k) > delta:
                            cnzwt += 1
                    nzw_scores[i].append(cnzw)
                    nzw_scores_delta[i].append(cnzwt)
                b = np.array(a[-1])
                for i in range(len(a[0])):
                    fis_scores[i].append(len(a[0][i]))
                temp_meta = []
                temp_meta.append(meta_explain.meta_avg(b))
                temp_meta.append(meta_explain.meta_median(b))
                temp_meta.append(meta_explain.meta_rule_based(a[0], a[2], b))
                for i in range(len(temp_meta)):
                    cnzw = 0
                    cnzwt = 0
                    for k in temp_meta[i]:
                        if abs(k) > 0:
                            cnzw += 1
                        if abs(k) > delta:
                            cnzwt += 1
                    nzw_scores[i+len(importance_test[j])].append(cnzw)
                    nzw_scores_delta[i+len(importance_test[j])].append(cnzwt)
                my_altruist.fis = len(meta_names)
                a = my_altruist.find_untruthful(X_test[j], temp_meta)
                for i in range(len(a[0])):
                    meta_scores[i].append(len(a[0][i]))
            count = 0
            row = [neural_name, noise, delta]
            for fis_score in fis_scores:
                row.append(np.array(fis_score).mean())
            for meta_score in meta_scores:
                row.append(np.array(meta_score).mean())
            with open('D2_'+neural_name+'.csv', 'a', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow(row)
            row = [neural_name, noise, delta]
            all_names = fi_names+meta_names
            for aname in range(len(all_names)):
                row.append(np.array(nzw_scores[aname]).mean())
                row.append(np.array(nzw_scores_delta[aname]).mean())
            with open('D2_NZW_'+neural_name+'.csv', 'a', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow(row)

Example of how to produce $f$ atom type arguments:

In [15]:
my_altruist = Altruist(neural_type, X_train, fi_techniques, feature_names,
                       level=noise, delta=delta, args=True)  # Enable args=True
# After running the find_untruthful method, the last element of the returned elements are the arguments for each interpretation technqiue
a = my_altruist.find_untruthful(X_test[j], importance_test[j])
arguments = a[-1]


Let's explore one argument:

In [16]:
arguments[0][1]  # 0 indicates the interpretation technique, and 1 the feature


['The alteration (INC) did not happen, as the feature value had the max value',
 "$f_{car, DEC}$: The evaluation of the alteration of $car$'s value to $0.9903$ ($DEC$) was performed and the model's behaviour was as expected $Increased$ (0.4386 to 0.4442), according to its importance $z_{car}=-0.63$."]

Ablation study

In [19]:
predict_functions = {'NN': predict_NN, 'cNN': predict_cNN}
for neural_name, neural_type in predict_functions.items():
    if neural_name == 'NN':
        fi_techniques = [fi_LRP_NN, fi_lime, fi_random]
        fi_names = ['LRP', 'LIME', 'RAND']
    else:
        fi_techniques = [fi_LRP_cNN, fi_lime, fi_random]
        fi_names = ['LRP', 'LIME', 'RAND']
    meta_names = ['Average', 'Median', 'RuleBased']

    with open(weights_path+'D2_'+neural_name+'.txt') as json_file:
        importances = json.load(json_file)
    importance_train = np.array(importances['train'])
    importance_test = np.array(importances['test'])
    meta_explain = MetaExplain(importance_train, feature_names)
    importance_train = np.delete(importance_train, 3, axis=1)
    importance_test = np.delete(importance_test, 3, axis=1)

    print('Starting evaluation for', neural_name)
    for noise in ['normal']:
        for delta in [0.0001]:
            my_altruist = Altruist(neural_type, X_train, fi_techniques,
                                   feature_names, prolog=False, level=noise, delta=delta)
            fis_scores = []
            meta_scores = []
            nzw_scores = []
            nzw_scores_delta = []
            for i in fi_techniques:
                fis_scores.append([])
                nzw_scores.append([])
                nzw_scores_delta.append([])
            for i in meta_names:
                meta_scores.append([])
                nzw_scores.append([])
                nzw_scores_delta.append([])
            for j in range(len(X_test)):
                my_altruist.fis = len(fi_techniques)
                a = my_altruist.find_untruthful(X_test[j], importance_test[j])
                for i in range(len(importance_test[j])):
                    cnzw = 0
                    cnzwt = 0
                    for k in importance_test[j][i]:
                        if abs(k) > 0:
                            cnzw += 1
                        if abs(k) > delta:
                            cnzwt += 1
                    nzw_scores[i].append(cnzw)
                    nzw_scores_delta[i].append(cnzwt)
                b = np.array(a[-1])
                for i in range(len(a[0])):
                    fis_scores[i].append(len(a[0][i]))
                temp_meta = []
                temp_meta.append(meta_explain.meta_avg(b))
                temp_meta.append(meta_explain.meta_median(b))
                temp_meta.append(meta_explain.meta_rule_based(a[0], a[2], b))
                for i in range(len(temp_meta)):
                    cnzw = 0
                    cnzwt = 0
                    for k in temp_meta[i]:
                        if abs(k) > 0:
                            cnzw += 1
                        if abs(k) > delta:
                            cnzwt += 1
                    nzw_scores[i+len(importance_test[j])].append(cnzw)
                    nzw_scores_delta[i+len(importance_test[j])].append(cnzwt)
                my_altruist.fis = len(meta_names)
                a = my_altruist.find_untruthful(X_test[j], temp_meta)
                for i in range(len(a[0])):
                    meta_scores[i].append(len(a[0][i]))
            count = 0
            row = [neural_name, noise, delta]
            for fis_score in fis_scores:
                row.append(np.array(fis_score).mean())
            for meta_score in meta_scores:
                row.append(np.array(meta_score).mean())
            with open('D2_Ablation_Study_'+neural_name+'.csv', 'a', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow(row)
            row = [neural_name, noise, delta]
            all_names = fi_names+meta_names
            for aname in range(len(all_names)):
                row.append(np.array(nzw_scores[aname]).mean())
                row.append(np.array(nzw_scores_delta[aname]).mean())
            with open('D2_Ablation_Study_NZW_'+neural_name+'.csv', 'a', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow(row)


Starting evaluation for NN
Results for: Noise -> normal  Delta -> 0.0001
	 NZW:
Starting evaluation for cNN
Results for: Noise -> normal  Delta -> 0.0001
	 NZW:
